In [1]:
!pip install ipython-autotime
%load_ext autotime
from prettytable import PrettyTable
from catboost import CatBoostRegressor
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import gc
import numpy as np
import pandas as pd
import timeit
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import lightgbm as lgb
from tqdm.notebook import tqdm
import pickle
import datetime
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype
import statsmodels.api as sm
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
from google.colab import drive

# get the default figure size
sns.set(rc={'figure.figsize':(15,5)})
path = './drive/My Drive/data/case_study_1/models/'
dir_ = './drive/My Drive/data/case_study_1/'

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=a834d993ac204ca0600eea35f291d1e845f95fde0a7e607b51d32a0a815bf2ed
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
     |████████████████████████████████| 65.9MB 54kB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def mount():
  '''Mount the google drive to current file system'''
  drive.mount('/content/drive')
def mount():
  '''Mount the google drive from current file system'''
  drive.flush_and_unmount()
mount()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
time: 1min 10s


# Utility Functions

In [3]:
def save_object(obj, filename):
  """Save the object into the disk"""
  
  dirname = os.path.dirname(filename)
  if not os.path.exists(dirname):
    os.makedirs(dirname)
  
  with open(filename, 'wb') as output: 
    pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
  """Load the object into the disk"""
  with open(filename, 'rb') as output:
    return pickle.load(output)

def add_time_features(df):
    """Add date time features by parsing timestamp"""
    # add dayofyear column
    df["dayofyear"] = df.timestamp.dt.dayofyear
    # add day column
    df["day"] = df.timestamp.dt.day
    # add week column
    df["weekday"] = df.timestamp.dt.weekday
    # add hour column
    df["hour"] = df.timestamp.dt.hour
    # add month column
    df["month"] = df.timestamp.dt.month
    # add weekend column
    df["weekend"] = df.timestamp.dt.weekday.apply(lambda x: 0 if x <5 else 1)

def free(obj_list):
  '''Remove unwanted objects to save memory'''
  # deleted the objects
  del obj_list
  # call the grabage collector
  gc.collect()

def merge_all(df, weather_df, building_meta_df):
  """Merge all data"""
  # merge train and bulding meta data on building id
  merged = df.merge(building_meta_df,
                    how= "left",
                    on= ["building_id"])
  
  # merge last merged df with weather data
  df_merged = merged.merge(weather_df, how= "left",
                           on= ["site_id", "timestamp"])
  # clean up
  free([df, weather_df, building_meta_df, merged])

  return df_merged

def interpolate_linear(df, column_list, n_site):
  '''Impute missing values using linear interpolation in both direction'''
  for i in range(n_site):
    # for each column and current site id, impute missing values
    for col in column_list:
      df.loc[df.site_id == i,  col] = df.loc[df.site_id == i, col].interpolate(method ='linear', limit_direction ='both')

def missing_info(df):
  """Print missing rows and percentage for each feature"""
  # number of null rows
  df.isna().sum()

  # get the count of nan value rows columnwise
  df_null_count = df.isna().sum().reset_index()
  df_null_count.rename(columns={"index": "column_name", 0: "missing_row_count"}, inplace= True)
  # alter the column name
  # compute the percentage and add new percentage column
  df_null_count["missing_row_percentage"] = df_null_count.missing_row_count/df.shape[0]
  df_null_count

# Original code from  https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    # print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    # print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    # print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

# number of sites
n_site = 16

time: 123 ms


# Training lightGBM
### Hyper parameter searching for each location
As dataset is huge, we will take 1000000 datapoint and features which we found useful in feature selection phase. For each location and grid search to find best hyper parameters.

After hyper parameter search, we will train 16 * 2 models (16 for each site (there are total 16 sites) and 2 cv set for each of 16 models) on the best hyperparameter values.

In [6]:
# supress userwarning
warnings.filterwarnings("ignore", category=UserWarning)

# location to store all our outputs
folder = "removed_zero_reading_and_different_hyper_parameter_for_each_location"

# load the unique value list from the disk
primary_use_unique_list = load_object(dir_ + "primary_use_unique_list.pkl")

# load the features which has been selected as a part of feature selection
selected_feature_list = ["meter_reading"] + list(load_object(dir_ + "selected_feature_list.pkl"))

# convert non-numeric to numeric
le = LabelEncoder()
le.fit(primary_use_unique_list)

# list of categorical features
categorical_features = [
    "building_id",
    "primary_use",
    "meter",
    "weekday",
    "hour"]

# parameters and corresponding values to search
search_parameters = {'num_leaves': (5, 10, 30, 50, 70, 100),
                    "bagging_fraction": (0.5, 0.8),
                    "feature_fraction": (0.5, 0.8),
                    "bagging_freq": (5, 10, 15)}

n = 1000000
# for each location data
for site_id in tqdm(range(n_site)):
  
  # read the dataframe of current site_id from the disk
  train_df = pd.read_feather(dir_+ f"siteWiseFeatureEngineeredTrainData/site_id_{site_id}/site_id_{site_id}.feather")

  # sample 1000000 points randomly and append it to the daatframe
  
  if train_df.shape[0] >= n:
    train_df = train_df.sample(n=n)

  # drop all rows where meter reading is 0
  '''
  This helped me reduce evaluation loss metric by around 0.1 in both public and private leaderboard
  '''
  train_df.drop(train_df[train_df.meter_reading == 0].index, axis="index", inplace= True)
  
  # choose only selected features
  train_df = train_df[selected_feature_list].copy(deep= True)

  # train features
  X = train_df.drop(["site_id", "timestamp", "meter_reading"], axis= "columns")
  
  # convert to numeric
  X["primary_use"] = le.transform(X["primary_use"])
  
  # train lables
  y = train_df["meter_reading"].copy(deep= True)

  # catboost regressor object
  lgb_reg = lgb.LGBMRegressor(learning_rate= 0.05)

  # grid search object
  grid_search_obj = GridSearchCV(lgb_reg, search_parameters, cv= 2,  return_train_score=True, verbose=0)

  # start grid search
  grid_search_obj.fit(X, y, categorical_feature= categorical_features)

  # save grid search results
  save_object(grid_search_obj, dir_ + f"girdSearchTrainModel/lightgbm/{folder}/grid_search_obj_site_id_{site_id}.pkl")

  # free up memory
  free([train_df, grid_search_obj])


time: 4h 49min 48s


### Training 16 models with 2 fold cross validation
Here, we will train lightGBM for each location data with 2 fold cross validation. So we will have 16 * 2 model. To get prediction on query point, we will use model corresponding to site_id as given in the query point and take average of output by both models(we are using 2 models for each site_id as cv set is 2)

In [9]:
folder = "removed_zero_reading"
grid_search_obj = load_object(dir_ + f"girdSearchTrainModel/lightgbm/{folder}/grid_search_obj_site_id_{10}.pkl")

time: 281 ms


In [ ]:
folder = "removed_zero_reading_and_different_hyper_parameter_for_each_location"
'''
For each location we perform:
  1) Load the data from the disk
  2) Select features listed in selected_feature_list
  3) Remove redundent features
  4) Encode categorical features (If needed)
  5) Train the model on best hyper parameter found
  6) Save the model into the disk
'''

# dictionary to hold site_id -> model object mapping
models = defaultdict(list)

# holds site_id and cv score 
cv_scores = {"site_id": [], "cv_score": []}

# load the unique value list from the disk
primary_use_unique_list = load_object(dir_ + "primary_use_unique_list.pkl")

# convert non-numeric to numeric
le = LabelEncoder()
le.fit(primary_use_unique_list)

# load the features which has been selected as a part of feature selection
selected_feature_list = ["meter_reading"] + list(load_object(dir_ + "selected_feature_list.pkl"))

# number of cv split
cv = 2

# random state seed
seed= 42

# for each location data
for site_id in tqdm(range(n_site)):
  # load grid search object 
  grid_search_obj = load_object(dir_ + f"girdSearchTrainModel/lightgbm/{folder}/grid_search_obj_site_id_{site_id}.pkl")

  # get best hyper parameter value
  bagging_fraction = grid_search_obj.best_params_["bagging_fraction"]
  bagging_freq = grid_search_obj.best_params_["bagging_freq"]
  feature_fraction = grid_search_obj.best_params_["feature_fraction"]
  num_leaves = grid_search_obj.best_params_["num_leaves"]
  
  # read the dataframe of current site_id from the disk
  train_df = pd.read_feather(dir_+ f"siteWiseFeatureEngineeredTrainData/site_id_{site_id}/site_id_{site_id}.feather")
  
  # drop all rows where meter reading is 0
  '''
  This helped me reduce evaluation loss metric by around 0.1 in both public and private leaderboard
  '''

  train_df.drop(train_df[train_df.meter_reading == 0].index, axis="index", inplace= True)
  
  # choose only selected features
  train_df = train_df[selected_feature_list].copy(deep= True)
  
  free(train_df)
  
  # free up memory
  reduce_mem_usage(train_df)

  # list of categorical features
  categorical_features = [
      "building_id",
      "primary_use",
      "meter",
      "weekday",
      "hour"]


  # train features
  X = train_df.drop(["site_id", "timestamp", "meter_reading"], axis= "columns")
  
  # convert to numeric
  X["primary_use"] = le.transform(X["primary_use"])
  
  # train lables
  y = train_df["meter_reading"].copy(deep= True)

  # holds prediction by our model
  y_pred_train_site = np.zeros(X.shape[0])

  # holds mean RMSE by our current model
  score = 0
  # original source of  K fold cross validation: https://www.kaggle.com/rohanrao/ashrae-divide-and-conquer

  kf = KFold(n_splits=cv, shuffle= True, random_state=seed)
  
  # for each fold, train model K-1 fold and test it on Kth fold
  for fold, (train_index, valid_index) in enumerate(kf.split(X, y)):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    # lightGBM regressor
    lgb_reg = lgb.LGBMRegressor(learning_rate= 0.05,
                                bagging_fraction= bagging_fraction,
                                bagging_freq= bagging_freq,
                                feature_fraction= feature_fraction,
                                num_leaves= num_leaves)
    # train the model
    lgb_reg.fit(X_train, y_train,
                eval_set= (X_valid, y_valid),
                eval_metric= "rmse",
                early_stopping_rounds= 25)

    # save the model instance in the dict
    models[site_id].append(lgb_reg)

    y_pred_valid = lgb_reg.predict(X_valid, num_iteration=lgb_reg.best_iteration_)
    y_pred_train_site[valid_index] = y_pred_valid

    rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
    print("Site Id:", site_id, ", Fold:", fold+1, ", RMSE:", rmse)
    score += rmse / cv
    
    gc.collect()
      
  cv_scores["site_id"].append(site_id)
  cv_scores["cv_score"].append(score)

  # free up the memory
  free([train_df])

  print("\nSite Id:", site_id, ", CV RMSE:", np.sqrt(mean_squared_error(y, y_pred_train_site)), "\n")

# save the model dict
save_object(models, dir_ + f"models/lightgbm/{folder}/models_dict.pkl")
save_object(cv_scores, dir_ + f"models/lightgbm/{folder}/cv_scores_dict.pkl")

# Prediction on test data
Use the traind model for each site_id and get the prediction for the same

In [15]:
# load the unique value list from the disk
primary_use_unique_list = load_object(dir_ + "primary_use_unique_list.pkl")

# number of cv split
cv = 2

# convert non-numeric to numeric
le = LabelEncoder()
le.fit(primary_use_unique_list)

# load the features which has been selected as a part of feature selection
selected_feature_list = ["meter_reading"] + list(load_object(dir_ + "selected_feature_list.pkl"))

# load the saved mode dict
# models = load_object(dir_ + f"models/lightgbm/{folder}/models_dict.pkl")

# holds ith prediction by our model
# 41697600 is total number of rows in the test data
y_pred_test_site = np.zeros(41697600)

# list of test years
year_list = [2017, 2018]

# for each location in test data
for site_id in tqdm(range(n_site)):
  # for each year
  for year in year_list:
    # read the dataframe of current site_id from the disk
    test_df = pd.read_feather(dir_+ f"siteWiseFeatureEngineeredTestData/{year}/site_id_{site_id}/site_id_{site_id}.feather")
    test_df.drop(["site_id", "timestamp"], axis= "columns", inplace= True)
    
    # get model of current location
    model = models[site_id]
    
    # list of row IDs
    row_id_list = test_df.pop("row_id")

    # free up memory
    reduce_mem_usage(test_df)

    # convert to numeric representation
    test_df["primary_use"] = le.transform(test_df["primary_use"])
    
    # get predictions by both model
    for i in range(cv):
      pred = model[i].predict(test_df)/cv
      y_pred_test_site[row_id_list] += pred

# prepare dataframe for submission
df_submit = pd.DataFrame({"row_id": range(41697600), "meter_reading": y_pred_test_site})

# undo log 1 plus operation and clip predictions (make it 0 if prediction is negative)
df_submit.meter_reading = np.clip(np.expm1(df_submit.meter_reading), 0, a_max=None)

# save the log predictions by our model
# save_object(y_pred_test_site, dir_ + "lightGBM/predictions/y_pred_test_site.pkl")

# df_submit.to_feather(dir_ + "df_submit.feather")
# df_submit.to_feather(dir_ + f"models/lightgbm/{folder}/submit.feather")
df_submit.to_csv(dir_ + f"models/lightgbm/{folder}/submit.csv", index= False)

# flush the stream and unmount the file system
unmount()


time: 12min 36s


# Experimentation and Results
Here is the brief information about experiments and results so far

* Initial submission gave us loss metric value of around  **_1.66_** and **1.44**. Here we have used global hyper parameter search (same hyper parameter for all 16 models)
<img src= "https://i.imgur.com/pjfWxbk.png" height= "300px"/>

* Removing the 0 value target rows(removing rows where meter_reading is 0 ) helped me to reduce evaluation loss metric by around  **_0.1_** in both public and private leaderboard
<img src= "https://i.imgur.com/MlFUfj6.png" height= "300px"/>

* Hyper parameter searching for each location seperatly (16 models with 2 fold cv set) made very little difference, which might not matter at all in real life problems. Here, we get around **_0.01_** loss value improvement at public score board and  **_0.06_** loss value improvement at private score
<img src= "https://i.imgur.com/3divqg4.png" height= "300px"/>
